# Condensation of water from humid air around a droplet

Description

    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 10/11/2020
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia



## Derivation of equations

Heat transfer between the vapor and air through a spherical interface of radius $r_v$ around each droplet of radius $d$;

$$
q=\frac{2 \pi k \left( T - T_a \right) N_d}{\rho_m \left( \frac{2}{d} - \frac{1}{r_v} \right)}
$$

$k$ is the thermal conductivity

$T$ is the liquid and vapor temperature, assumed equal

$T_a$ is the air temperature

$\rho_m$ is the mixture density

$N_d$ is the number density of liquid droplets

This should be equal to the heat transfer into the cold vapor and liquid;

$$
q = \left( x c_{pv} + (1-x) c_{pl} \right) (1-y) \frac{dT}{dt}
$$

$c_{pv}$ is the vapor heat capacity

$c_{pl}$ is the liquid heat capacity

$x$ is the vapor mass fraction (to total vapor+liquid)

$y$ is the air mass fraction (to total mass)

$t$ is time

This must also be equal to the heat transfer into the air and water;

$$
q = y c_{pa} \frac{dT_a}{dt} + \frac{c_{pw}}{\rho_m V} \left[ \frac{d m_c}{dt} \left( T_a - T_d \right) + m_c \frac{dT_a}{dt} \right] + \frac{h_{fgw}}{\rho_m V} \frac{d m_c}{dt}
$$

$c_{pa}$ is the air heat capacity

$c_{pw}$ is water heat capacity

$V$ is the cell volume

$h_{fgw}$ is the latent heat of water

$m_c$ is the mass of condensed liquid water

$T_d$ is the dew point temperature of the air

The rate of captured liquid water condensation $\frac{d m_c}{dt}$ is determined by mass diffusion from the air into the vapor;

$$
\frac{d m_c}{dt} = \frac{ 4D \pi r_v^2 p_w \rho_w \rho_v y N_d V} {3 p \rho_a}
$$

$D$ is the diffusion coefficient

$p_w$ is the partial pressure of the water in the air

$p$ is the absolute pressure

$\rho_w$ is the water density

$\rho_v$ is the vapor density

$\rho_a$ is the air density

We can further constrain the equations by requiring the radius of the vapor region $r_v$ to be such that the total volume of vapor matches the cell average value;

$$
r_v = \left[ \frac{ 3x(1-y)\rho_m }{4 N_d \pi \rho_v} \right]^{1/3}
$$

## Solution method

Require all heat transfer terms be equal. Assuming all the thermophysical and local constants can be obtained, this leads to 5 equations above with 7 unknown variables: $r_v$, $q$, $T_a$, $\frac{dT}{dt}$, $\frac{dT_a}{dt}$, $\frac{d m_c}{dt}$, $m_c$.  

We can solve this system as an IVP with the initial values set by the simulation values. We define the state vector:

$$
\psi = \begin{bmatrix} T \\ T_a \\ m_c \end{bmatrix}
$$

Then we can solve the nonlinear ODE of the form

$$
\psi' = \begin{bmatrix} \frac{dT}{dt} \\ \frac{dT_a}{dt} \\ \frac{dm_c}{dt} \end{bmatrix} = f \left( t, \psi \right)
$$

We solve $f$ by determining the values of $\frac{dT}{dt}$, $\frac{dT_a}{dt}$ and $q$ that satisfy the first three equations above for specified values of $T$, $T_a$ and $m_c$ which at $t=0$ are determined by the simulation cell average, and $m_c(t=0)=0$. The derivatives are then fed back into a Runge-Kutta solver. The problem is integrated until the end of the cell integration time $\Delta t$.



In [48]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize, scipy.integrate
import sys
%matplotlib notebook

In [2]:
# Constants
k_air = lambda T: 7.940e-5*T + 2.340e-3 # W/m-K in air [Incropera et al]
Cpa = lambda T: 2.667e-6*(T**3) - 1.8e-3*(T**2) + 4.233e-1*T + 9.72e2 # kJ/kg-K for air [Incropera et al]
rho_a = lambda T: 3.5783e2 * T**(-1.0047)  # kg/m3 for air [Incropera et al]

kv = lambda T: 8.4569e-5*T - 1.1409e-2 # W/m-K in R-134a vapor [Laesecke et al 1992]
Cpv = lambda T: 102.03*(1.94006e1 + 2.58531e-1*T + -1.29665e-4*T**2) # J/kg-K for R-134a vapor [Dupont]
Cpl = lambda T: 5.2419e-3*(T**2) - 1.6133*T + 8.6750e2 # J/kg-K for R-134a saturated liquid at 1 atm [Dupont]

rho_w = lambda T: 1.5733e-4*(T**3) - 1.4079e-1*(T**2) + 4.1748e1*T - 3.1054e3 # kg/m3 for subcooled water [Hare & Sorensen]
Cpw = lambda T: -1.1930e-2*(T**3) + 1.0037e1*(T**2) - 2.8151e3*T + 2.6740e5 # kJ/kg-K for water [Angell 1982]
hfg = lambda T: 2500.8 -2.36*(T-273.15) + 0.0016*(T-273.15)**2 - 0.00006*(T-273.15)**3 # kJ/kg for water [Rogers & Yau 1989]

D = 2.82e-5 # m^2/s  (water in air)

In [3]:
# Antoine Equation for vapor pressure
#    T [K]
#    RH [0-1]
def Pv(T,RH):
    return 1e5*RH*np.exp(4.6543-(1435.264/(T-64.848)))

In [4]:
# Magnus dew point temperature
#    T [K]
#    RH [0-1]
def Tdew(T,RH):
    return 273.15 + (257.14*np.log(Pv(T,RH)/611.21)/(18.678-np.log(Pv(T,RH)/611.21)))

In [5]:
# Model function for radius of vapor field around the droplets
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_m = mixture density [kg/m^3]
#     Nd = number density [m^-3]
#     rho_v = vapor density [kg/m^3]
def frv(x,y,rho_m,Nd,rho_v):
    f1 = 3*x*(1-y)*rho_m
    f2 = 4*Nd*np.pi*rho_v
    return (f1/f2)**0.33333

In [6]:
# Function for vapor-air heat transfer
#     T = liquid-vapor equilibrium temperature [K]
#     Tair = air temperature [K]
#     Nd = number density [m^-3]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_v = vapor density [kg/m^3]
#     rho_m = mixture density [kg/m^3
#     returns: heat transfer [W]
def fheat(T,Tair,Nd,d,x,y,rho_v,rho_m):
    f1 = 4*np.pi*k_air(Tair)*(T-Tair)*Nd
    f2 = rho_v*((2./d) - (1./frv(x,y,rho_m,Nd,rho_v)))
    return (f1/f2)

In [7]:
# Function for vapor-liquid rate change of enthalpy
#     q = heat transfer [W]
#     dTdt = dT/dt, rate change of liquid-vapor temperature [K/s]
#     T = liquid-vapor equilibrium temperature [K]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
def fhlv(q,dTdt,x,y,T):
    return (x*Cpv(T)+(1-x)*Cpl(T))*(1-y)*dTdt

In [8]:
# Function for water mass transport rate using Fick's Law
#     D = Diffusion coefficient [m^2/s]
#     x = mass fraction vapor = m_vap / m_liq
#     y = air mass fraction = m_a / m
#     rho_m = mixture density [kg/m^3]
#     Nd = number density [m^-3]
#     rho_v = vapor density [kg/m^3]
#     Tair = air temperature [K]
#     RH = Relative humidity [0-1]
#     V = cell volume [m^3]
#     p = absolute pressure [Pa]
def dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p):
    r_v=frv(x,y,rho_m,Nd,rho_v)
    f1 = 4*D*np.pi*(r_v**2)*Pv(Tair,RH)*rho_w(Tair)*y*rho_v*Nd*V
    f2 = 3*p*rho_a(Tair)
    return f1/f2

In [9]:
# Function for air and water vapor rate change of enthalpy
#     dTadt = dTair/dt, rate change of air temperature [K/s]
#     rho_m = mixture density [kg/m^3
#     Nd = number density [m^-3]
#     Tair = air temperature [K]
#     mc = mass of condensed water [kg]
#     RH = Relative humidity [0-1]
#     p = absolute pressure [Pa]
#     returns: heat transfer [W]
def fhaw(dTadt,rho_m,V,Tair,mc,RH,p,D):
    dmc_dt = dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p)
    f1 = y*Cpa(Tair)*dTadt
    f2 = (Cpw(Tair)/(rho_m*V))*dmc_dt*(Tair-Tdew(Tair,RH))
    f3 = (Cpw(Tair)/(rho_m*V))*mc*dTadt
    f4 = hfg(Tair)*dmc_dt/(rho_m*V)
    return f1 + f2 + f3 + f4

In [64]:
# Constants from CFD.
Tlv=247.
Tair0=297.
V=1e-12
Nd=10**12.12
d=5.0118e-6
x=1e-3
y=.5
rho_v=1.6
rho_m=4.1
RH=1.0
p=1e5

# Solve dTdt and dTadt such that q1,q2,q3 are all equal
def zeroq_fun(X,Y,*args):
    
    #print(X)
    dTdt,dTadt,q=X
    T,Tair,mc=Y
    
    Nd,d,x,y,rho_v,rho_m,V,RH,p,D=args
    
    # Solve dTdt and dTadt such that q1,q2,q3 are all equal
    
    q1 = -fheat(T,Tair,Nd,d,x,y,rho_v,rho_m)*1e-3 # vapor-air heat transfer [kW]
    q2 = fhlv(T,dTdt,x,y,T)*1e-3 # enthalpy change in the liquid-vapor mixture [kW]
    q3 = -fhaw(dTadt,rho_m,V,Tair,mc,RH,p,D)*1e-3 # enthalpy change in the moist air [kW]

    #print(dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p))
    
    return (q1-q)**2, (q2-q)**2, (q3-q)**2  # all should go to zero so that q1==q2==q3



#scipy.optimize.fsolve(zeroq_fun,x0=(Tair-T,T-Tair,0),args=((T,Tair,mc),Nd,d,x,y,rho_v,rho_m,V,RH,p,D),maxfev=10000)

In [65]:
# dY/dt = f(t,Y,...)
def deriv_fun(t,Y,args):
    
    #print(t,Y,args)
    T,Tair,mc=Y
    
    Nd,d,x,y,rho_v,rho_m,V,RH,p,D=args
    
    X = scipy.optimize.fsolve(zeroq_fun,x0=(Tair-T,T-Tair,0),\
                              args=((T,Tair,mc),Nd,d,x,y,rho_v,rho_m,V,RH,p,D),\
                              maxfev=1000)
    
    dTdt,dTadt,q = X
    
    dmc_dt = dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p)
    
    return [dTdt, dTadt, dmc_dt]

solv = scipy.integrate.ode(deriv_fun)
solv.set_initial_value([Tlv,Tair0,0],0)
solv.set_f_params((Nd,d,x,y,rho_v,rho_m,V,RH,p,D))
dt=1e-4
Yprev=[0, 0, 0]
while solv.successful() and solv.t < 0.01:
    Y=solv.integrate(solv.t+dt)
    # dmc_dt converged to within 0.5% of equilibrium before time ran out?
    if (solv.t>1e-3) and (((Y[-1]-Yprev[-1])/Yprev[-1])<=0.005):
        break
    Yprev=Y
    #print(solv.t+dt, Y, dY[-1])
    #sys.stdout.flush()

T,Tair,mc=Y
print("End result dmc/dt")
print(solv.t)
print(dmcdt(D,x,y,rho_m,Nd,rho_v,RH,Tair,V,p))

End result dmc/dt
0.010099999999999994
4.2067006439872156e-13
